In [16]:
from datetime import datetime
import yfinance as yf
import pandas as pd
from IPython.display import display, HTML
from prettytable import PrettyTable  
import ssl
pd.options.display.float_format = '{:.3f}'.format

In [17]:
def riskFreeRate():
    url_rf = 'https://home.treasury.gov/resource-center/data-chart-center/interest-rates/TextView?type=daily_treasury_yield_curve&field_tdr_date_value_month=202207'
    dfs_rf = pd.read_html(url_rf, attrs={"class":"views-table"}, index_col='Date')
    df_rf = dfs_rf[0]
    risk_free_rate = df_rf.iloc[-1,16] /100
    return risk_free_rate

In [18]:
#def equityRiskPremium():
#    url_erp = 'https://www.kroll.com/-/media/cost-of-capital/kroll-us-erp-rf-table-2022.pdf'
#    dfs_erp = tabula.read_pdf(url_erp, pages='all',  lattice=False)
#    df_erp = dfs_erp[0]
#    df_erp = df_erp[['Unnamed: 4']]
#    df_erp = df_erp['Unnamed: 4'].str.split('\r',expand=True)
#    erp = pd.to_numeric(df_erp.iloc[0,1]) / 100
#    return erp

# alternate way if tabula doesn't work
def equityRiskPremium():
    ssl._create_default_https_context = ssl._create_unverified_context
    url_rf = 'https://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/ctryprem.html'
    dfs_rf = pd.read_html(url_rf)
    df_rf = dfs_rf[0]
    new_header = df_rf.iloc[0] #grab the first row for the header
    df_rf = df_rf[1:] #take the data less the header row
    df_rf.columns = new_header #set the header row as the df header
    df_rf.set_index('Country', inplace=True)
    df_rf.columns = ["Moody's Rating", "Adj Default Spread", "Country Risk Premium", "Equity Risk Premium", "Country Risk Premium"]
    df_rf['Equity Risk Premium'] = df_rf['Equity Risk Premium'].str.rstrip('%').astype('float')
    erp = df_rf.loc['United States']['Equity Risk Premium'] / 100
    return erp
#print(erp)

In [19]:
def stockBio(symbol):
    dfs = []
    stock = yf.Ticker(symbol)
    stock_bio = stock.info
    stock_df = pd.DataFrame(list(stock_bio.items()))
    stock_df.columns = ['key','value']
    stock_df.set_index("key", inplace = True)
    df_financials = stock.financials
    df_financials.apply(pd.to_numeric)
#    dfs.append(stock_df)
#    dfs.append(df_financials)
 #   return dfs
    myDict = {}
 #   stock_df = stockBio(symbol)[0]
 #   df_financials = stockBio(symbol)[1]
    myDict['stock_beta'] = stock_df.loc['beta'].values[0]
    myDict['interest_expense'] = abs(df_financials.iloc[10,0])
    myDict['total_debt'] = stock_df.loc['totalDebt'].values[0]
    myDict['income_tax_expense'] = abs(df_financials.iloc[14,0])
    myDict['income_before_tax'] = df_financials.iloc[2,0]
    myDict['enterprise_value'] = stock_df.loc['enterpriseValue'].values[0]
    myDict['freeCashflow'] = stock_df.loc['freeCashflow'].values[0]
    myDict['total_cash'] = stock_df.loc['totalCash'].values[0]
    myDict['shares_outstanding'] = stock_df.loc['sharesOutstanding'].values[0]
    myDict['current_price'] = stock_df.loc['currentPrice'].values[0]
    myDict['company'] = stock_df.loc['longName'].values[0]
    return myDict

In [20]:
def getCalcData():
    holder = stockBio # prevents repeated calls to the api
    myDictCalc = {}
    myDictCalc['cost_of_equity'] = ((holder['stock_beta'] * (equityRiskPremium())) + (riskFreeRate()))
    myDictCalc['cost_of_debt_preTax'] = (holder['interest_expense'] / holder['total_debt'])
    myDictCalc['tax_rate'] = (holder['income_tax_expense'] / holder['income_before_tax'])
    myDictCalc['cost_of_debt_postTax'] = myDictCalc['cost_of_debt_preTax'] * (1 - myDictCalc['tax_rate'])
    myDictCalc['debt_to_cap'] = holder['total_debt'] / holder['enterprise_value']
    myDictCalc['WACC'] = (1 - myDictCalc['debt_to_cap']) * myDictCalc['cost_of_equity'] + (myDictCalc['debt_to_cap'] * myDictCalc['cost_of_debt_postTax'])
    return myDictCalc

In [21]:
def growthRate(growth_rate):
    holderGrowth = calcData
  #  growth_rate = growth_rate
  #  growth_rate = float(input('Enter the expected growth rate: '))
    if growth_rate > holderGrowth['WACC']:
        print('Growth rate cannot exceed WACC, reverting to default values')
        growth_rate = calcData['WACC'] - riskFreeRate()
    else:
        growth_rate = growth_rate
        
    return growth_rate
        
        

In [22]:
def getNPV():
    holder = stockBio
    holderNPV = calcData
    currentYear = datetime.now().year
    years_list = []
    freecashflow_list = []
    discount_factor_list = []
    for i in range(1,7):
        years_list.append(int(currentYear + i - 1))
        freecashflow_list.append(holder['freeCashflow'] * ((1 + growth_rate) ** i))
    for i in range(1,6):
        discount_factor_list.append(1/((1 + holderNPV['WACC']))**i)
    discount_factor_list.append(1/((1 + holderNPV['WACC']))**5) # terminal year
    
    df_npv = pd.DataFrame(columns = ['years','free cash flow', 'discount factor','present value'])
    
    df_npv['years'] = years_list
    df_npv['free cash flow'] = freecashflow_list
    df_npv['discount factor'] = discount_factor_list
    df_npv['present value'] = df_npv['free cash flow'] * df_npv['discount factor']
    
    terminal_year_cashflow = df_npv['free cash flow'].iloc[-1]
    discount_factor_terminalYear = df_npv['discount factor'].iloc[-1]

    present_value_TerminalYear = (terminal_year_cashflow / (holderNPV['WACC'] - growth_rate)) * discount_factor_terminalYear
    #present_value_TerminalYear = (terminal_year_cashflow / .01 ) * discount_factor_terminalYear
    #df_npv.loc[df_npv['years'] == currentYear + 3, ['years']] = 'Terminal Year'
    df_npv['years'].iloc[-1] = 'Terminal Year'
    df_npv['present value'].iloc[-1] = present_value_TerminalYear
    df_npv.set_index('years', inplace=True)
    npv = df_npv['present value'].sum()
    
    return [npv, df_npv]
    

In [23]:
def equityValue():
    holderEQV = stockBio
    equity_value = getNPV()[0] + holderEQV['total_cash'] - holderEQV['total_debt']
    return equity_value

In [24]:
def shareValuation():
    holderSV = stockBio
    share_valuation = equityValue() / holderSV['shares_outstanding']
    return share_valuation

In [25]:
def printstockBio():
  #  df = pd.DataFrame(list(stockBio.items()), columns=['Key', 'Value'])
    df = PrettyTable(['Key', 'Value'])
    count = 0
    for key,value in stockBio.items():
        if count > 0 & count < 9:
            if (isinstance(value, int)) | (isinstance(value, float)):
                df.add_row([key,"${0:,.0f}".format(value)])
                count += 1
            else:
                df.add_row([key,value])
                count += 1
        else:
            df.add_row([key,value])
         #   count += 1
    display(df)

In [26]:
def printcalcData():
    #df = pd.DataFrame(list(calcData.items()), columns=['Key', 'Value'])
    df = PrettyTable(['Key', 'Value'])
    count = 0   
    for key,value in calcData.items():
        if count <= 5:
            df.add_row([key,"{0:,.3f} %".format(value*100)])
            count += 1
        elif (count >= 6) & (count <= 8):
            df.add_row([key,"${0:,.2f}".format(value)])
            count += 1
        else:
            df.add_row([key,value])
    display(df)

In [27]:
def recommendation(share_valuation):
    if share_valuation > stockBio['current_price']:
        recommendation = 'BUY'
    elif (share_valuation >= stockBio['current_price'] * .90):
        recommendation = "HOLD"
    else:
        recommendation = "SELL"
    return recommendation

In [28]:

symbol = input('Enter the stock symbol: ')
growth_rate = float(input('Enter growth rate: '))
stockBio = stockBio(symbol)
calcData = getCalcData()
growth_rate = growthRate(growth_rate)
calcData['NPV'] = getNPV()[0]
calcData['Equity Value'] = equityValue()
calcData['Fair Value']= shareValuation()
calcData['Recommendation'] = recommendation(calcData['Fair Value'])
printstockBio()
printcalcData()
display(getNPV()[1])

Enter the stock symbol: pypl
Enter growth rate: 0.07


<ipython-input-18-232c747e337e>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rf['Equity Risk Premium'] = df_rf['Equity Risk Premium'].str.rstrip('%').astype('float')
C:\Users\Tushar\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Key,Value
stock_beta,1.433375
interest_expense,232000000.0
total_debt,10026999808
income_tax_expense,70000000.0
income_before_tax,4099000000.0
enterprise_value,94983127040
freeCashflow,1773624960
total_cash,7965000192
shares_outstanding,1158040064
current_price,80.97


Key,Value
cost_of_equity,8.988 %
cost_of_debt_preTax,2.314 %
tax_rate,1.708 %
cost_of_debt_postTax,2.274 %
debt_to_cap,10.557 %
WACC,8.279 %
NPV,"$148,401,211,874.84"
Equity Value,"$146,339,212,258.84"
Fair Value,$126.37
Recommendation,BUY


,free cash flow,discount factor,present value
years,,,
2022,1897778707.200,0.924,1752677738.981
2023,2030623216.704,0.853,1731977913.030
2024,2172766841.873,0.788,1711522560.312
2025,2324860520.804,0.727,1691308793.502
2026,2487600757.261,0.672,1671333759.372
Terminal Year,2661732810.269,0.672,139842391109.639
